In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
from scipy.stats import t
from scipy import stats
import xml.etree.ElementTree as ET
import requests

#Optional status bars
from tqdm import tqdm

def calculate_mean_dividend_growth(div):
    year_div = div.resample('Y').sum()
    div_pct = year_div.pct_change()
    filter_div_pct = div_pct[(div_pct <= 1.0) & (div_pct >= -1.0)]
    return filter_div_pct.mean() if len(filter_div_pct) > 0 else 0

#Pull data from the treasury on the current 10-year treasury rate
fed_scrape ='https://home.treasury.gov/sites/default/files/interest-rates/yield.xml'
response = requests.get(fed_scrape)  #we want to bootstrap
if response.status_code == 200:
    treas_xml = response.content
    root = ET.fromstring(treas_xml)
    subchildren = root.findall('.//AVERAGE_10YEAR')
    if subchildren:
        latest_subchild = subchildren[-1]
    else:
        print('unable to locate published average.')
        subchildren_1 = root.findall('.//BC_10YEAR')
        if subchildren_1:
            lastest_subchild = subchildren[-1]
        else:
            print('unable to find published data')
            latest_subchild = str(input('Enter manually'))
else:
    print(f"failed to retrieve treasury data. Status code: {response.status_code}")

rf = float(latest_subchild.text)/100

#Get an implied ERP estimate using the dividend growth model 
sp500 = "MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,BXP,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CF,CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CPAY,CTVA,CSGP,COST,CTRA,CRWD,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DAY,DECK,DE,DAL,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GE,GEHC,GEV,GEN,GNRC,GD,GIS,GM,GPC,GILD,GPN,GL,GDDY,GS,HAL,HIG,HAS,HCA,DOC,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUBB,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,INCY,IR,PODD,INTC,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,INVH,IQV,IRM,JBHT,JBL,JKHY,J,JNJ,JCI,JPM,JNPR,K,KVUE,KDP,KEY,KEYS,KMB,KIM,KMI,KKR,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LDOS,LEN,LLY,LIN,LYV,LKQ,LMT,L,LOW,LULU,LYB,MTB,MRO,MPC,MKTX,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,META,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,MOH,TAP,MPWR,MNST,MCO,MS,MOS,MSI,MSCI,NDAQ,NTAP,NFLX,NEM,NWSA,NWS,NEE,NKE,NI,NDSN,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,ON,OKE,ORCL,OTIS,PCAR,PKG,PANW,PARA,PH,PAYX,PAYC,PYPL,PNR,PEP,PFE,PCG,PM,PSX,PNW,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PEG,PTC,PSA,PHM,QRVO,PWR,QCOM,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RVTY,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SRE,NOW,SHW,SPG,SWKS,SJM,SW,SNA,SOLV,SO,LUV,SWK,SBUX,STT,STLD,STE,SYK,SMCI,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TRGP,TGT,TEL,TDY,TFX,TER,TSLA,TXN,TXT,TMO,TJX,TSCO,TT,TDG,TRV,TRMB,TFC,TYL,TSN,USB,UBER,UDR,ULTA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VLTO,VRSN,VRSK,VZ,VRTX,VTRS,VICI,V,VST,VMC,WRB,GWW,WAB,WBA,WMT,DIS,WBD,WM,WAT,WEC,WFC,WELL,WST,WDC,WY,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS"
sp500_tickers =sp500.split(',')
idx_dict = {}

#Uses the defined function to collect dividend data 
for tick in tqdm(sp500_tickers, desc="processing s&p tickers"):
    stock =yf.Ticker(tick)
    div=stock.dividends
    if not div.empty:
        mean_div_growth = calculate_mean_dividend_growth(div)
        idx_dict[tick] = mean_div_growth
    else:
        idx_dict[tick] = 0
        
#average growth on the collected series, then DGM
idx_div_df = pd.Series(idx_dict)
growth = idx_div_df.mean()
div_forward = (1 + growth) * idx_div_df.iloc[-1]
idx_quote = yf.Ticker('^GSPC').history(period='1d')['Close'].iloc[-1]
implied_erp = ((div_forward / idx_quote) + growth) - rf

processing s&p tickers: 100%|████████████████████████████████████████████████████████| 502/502 [04:15<00:00,  1.97it/s]


In [10]:
rap = []
USER_0 = True
while USER_0 == True:
    user_data = input("Type in Tickers to be Analyzed. Type 'DONE' when done.")
    if user_data == 'DONE':
        USER_0 = False
    else:
        rap.append(user_data)

start=input("Enter beginning date of search in the following format: '2019-01-01'")
end=input("Enter ending date of search in the following format: '2024-01-01'")
resample = 'MS'

USER_4 = 0
user_input = input('Type "YES" for short sales')
if user_input == "YES":
    USER_4 = 1
else:
    print('Short sales disabled.')

rap_dict = {}
rap_sigma = []
rap_alpha = []
rap_beta = []
rap_CI_plus =[]
rap_CI_minus = []
rap_info = []
rap_error = []

snp_df = yf.download('^GSPC', start=start, end=end, progress=False).pct_change().dropna() -rf
snp_df = snp_df.resample(resample).first()
sigma_x = snp_df['Close'].std()
x_bar = snp_df['Close'].mean()
n_x = len(snp_df['Close'])


for i in rap:
    df_target = yf.download(i, start=start, end=end, progress=False).pct_change().dropna() -rf
    df_target = df_target.resample(resample).first()
    sigma_y = df_target['Close'].std()
    y_bar = df_target['Close'].mean()
    n_y = len(df_target['Close'])
    pearson_xy = snp_df['Close'].corr(df_target['Close'])
    r_square = pearson_xy**2
    Beta_xy = pearson_xy*sigma_x/sigma_y 
    alpha_xy = y_bar - Beta_xy*x_bar
    Sum_Squares_Error = sum((snp_df['Close'] - (alpha_xy + Beta_xy*df_target['Close']))**2)
    Sum_Squares_Regression = sum(((alpha_xy + Beta_xy*df_target['Close']) - y_bar)**2)
    Mean_Square_Error = Sum_Squares_Error/n_y-2
    Mean_Square_Regression = Sum_Squares_Regression/1
    F_Stat = Mean_Square_Regression/Mean_Square_Error
    standard_error_estimate = Mean_Square_Error**.5

    if stats.f.sf(F_Stat,1,n_y-2) <= .05:
        beta_t = (Beta_xy-1)/(standard_error_estimate/sigma_x)
        alpha_t = (alpha_xy - 0)/((1/n_x)+(x_bar**2/sigma_x**2))**.5
    else:
        significance = False

    #Confidence Interval
    Standard_Error_Forecast = standard_error_estimate*((1+1/n_x+(implied_erp-x_bar)**2/sigma_x**2)**.5)
    Y_hat = alpha_xy +Beta_xy*implied_erp
    t_value = t.ppf(1-.025,n_y-2)
    CI_plus = Y_hat+t_value*Standard_Error_Forecast
    CI_minus =Y_hat-t_value*Standard_Error_Forecast
    info_ratio = alpha_xy/Sum_Squares_Error
    
    rap_sigma.append(sigma_x)
    rap_alpha.append(alpha_xy)
    rap_beta.append(Beta_xy)
    rap_CI_plus.append(CI_plus)
    rap_CI_minus.append(CI_minus)
    rap_info.append(info_ratio)
    rap_error.append(Sum_Squares_Error)
    
rap_dict = {'Ticker': rap, 'Stdev': rap_sigma, 'Alpha': rap_alpha, 'Beta' : rap_beta, 'CI "-"':
            rap_CI_minus, 'CI "+"': rap_CI_plus, 'info_ratio':rap_info, 'Error': rap_error}    
rap_df = pd.DataFrame(rap_dict)
rap_df.set_index('Ticker', inplace=True)

if USER_4 == 1:
    rap_df['weights'] = rap_df['info_ratio']/rap_df['info_ratio'].sum()
else: 
    rap_df['weights'] = rap_df['info_ratio']/rap_df['info_ratio'].sum().clip(min=0)
    #rap_df['weights'] = rap_df['weights'].clip(lower=0)

alpha_p = (rap_df['weights']*rap_df['Alpha']).sum()
error_p = (rap_df['weights']**2 * rap_df['Error']).sum()

initial_position = (alpha_p/error_p)/(implied_erp/(sigma_x**2))
beta_active = (rap_df['weights']*rap_df['Beta']).sum()
adj_position = initial_position/(1-(1-beta_active)*initial_position)
idx_weight = 1-adj_position
display = rap_df['weights']*adj_position
RP_optimal = (idx_weight+beta_active*adj_position)*implied_erp+adj_position*alpha_p
Variance_P = (idx_weight+beta_active*adj_position)**2*sigma_x**2+(adj_position**2*error_p)
print(display)
print("weight in passive portfolio:", idx_weight)
Sharpe_dowsed = RP_optimal/(Variance_P**.5)
Sharpe_passive = implied_erp/sigma_x
print("Sharpe W/Stocks:", Sharpe_dowsed)
print("Sharpe W/O Stocks:", Sharpe_passive)

Type in Tickers to be Analyzed. Type 'DONE' when done. MSFt
Type in Tickers to be Analyzed. Type 'DONE' when done. DONE
Enter beginning date of search in the following format: '2019-01-01' 2019-01-01
Enter ending date of search in the following format: '2024-01-01' 2024-01-01
Type "YES" for short sales YES


Ticker
MSFt   -0.041691
Name: weights, dtype: float64
weight in passive portfolio: 1.041690508632659
Sharpe W/Stocks: 1.3571779482986224
Sharpe W/O Stocks: 1.329560524975003
